# HOG-Based Face Detection Functions Documentation


This document explains each function used in the HOG-based face detection example code. The code demonstrates:
- Computing image gradients using different differential filters (Sobel, Prewitt, Scharr).
- Extracting Histogram of Oriented Gradients (HOG) features.
- Visualizing HOG features overlaid on the original image.
- Performing sliding window matching using Normalized Cross-Correlation (NCC) on HOG features.
- Applying Non-Maximum Suppression (NMS) to filter duplicate detections.
- Visualizing detected face regions with bounding boxes and NCC scores.

Below are the detailed explanations for each function.

#### Import Libraries

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy.signal import convolve2d

#### Differential Filter Implementation
**Description:**  
Computes the gradients of the input grayscale image using a differential filter. The function supports three methods: **Sobel**, **Prewitt**, and **Scharr**. Also, please use scipy convolve2d function for calculation.
- **Parameters:**
  - `image`: 2D numpy array (grayscale image).
  - `method`: String indicating the differential filter method ('sobel', 'prewitt', or 'scharr').
- **Returns:**
  - `grad_x`: The gradient of the image in the x-direction.
  - `grad_y`: The gradient of the image in the y-direction.
- **Usage:**  
  This function is used as the first step in HOG feature extraction to obtain the gradient magnitudes and orientations.


##### Sobel Operator

**Description:**  
The Sobel operator emphasizes intensity changes by combining smoothing and differentiation. It is widely used in edge detection and feature extraction.

**Kernels:**

- **Kernel for x-direction:**

  $$
  \begin{pmatrix}
  -1 & 0 & 1 \\
  -2 & 0 & 2 \\
  -1 & 0 & 1
  \end{pmatrix}
  $$

- **Kernel for y-direction:**

  $$
  \begin{pmatrix}
  -1 & -2 & -1 \\
   0 &  0 &  0 \\
   1 &  2 &  1
  \end{pmatrix}
  $$

##### Prewitt Operator

**Description:**  
The Prewitt operator approximates the gradient with simpler kernels and without the extra weighting used in the Sobel operator. It is computationally more efficient though slightly less robust against noise.

**Kernels:**

- **Kernel for x-direction:**

  $$
  \begin{pmatrix}
  -1 & 0 & 1 \\
  -1 & 0 & 1 \\
  -1 & 0 & 1
  \end{pmatrix}
  $$

- **Kernel for y-direction:**

  $$
  \begin{pmatrix}
  -1 & -1 & -1 \\
   0 &  0 &  0 \\
   1 &  1 &  1
  \end{pmatrix}
  $$

##### Scharr Operator

**Description:**  
The Scharr operator provides improved rotational symmetry and more accurate gradient estimation compared to Sobel. It applies stronger weighting to the central pixels, enhancing edge detection.

**Kernels:**

- **Kernel for x-direction:**

  $$
  \begin{pmatrix}
   -3 &  0 &  3 \\
  -10 &  0 & 10 \\
   -3 &  0 &  3
  \end{pmatrix}
  $$

- **Kernel for y-direction:**

  $$
  \begin{pmatrix}
   -3 & -10 & -3 \\
    0 &   0 &  0 \\
    3 &  10 &  3
  \end{pmatrix}
  $$

In [13]:
def compute_gradients(image, method='sobel'):
    """
    Compute image gradients using a differential filter.
    
    Parameters:
    - image: 2D numpy array (grayscale)
    - method: 'sobel', 'prewitt', or 'scharr'
    
    Returns:
    - grad_x: gradient in the x direction
    - grad_y: gradient in the y direction
    """
    if method.lower() == 'sobel':
        #############
        # CODE HERE #
        #############
        kernel_x = np.array([[-1, 0, 1],
                      [-2, 0, 2],
                      [-1, 0, 1]])
        kernel_y = np.array([[-1, -2, -1],
                      [0, 0, 0],
                      [1, 2, 1]])
    elif method.lower() == 'prewitt':
        #############
        # CODE HERE #
        #############
        kernel_x = np.array([[-1, 0, 1],
                      [-1, 0, 1],
                      [-1, 0, 1]])
        kernel_y = np.array([[-1, -1, -1],
                      [0, 0, 0],
                      [1, 1, 1]])
    elif method.lower() == 'scharr':
        #############
        # CODE HERE #
        #############
        kernel_x = np.array([[-3, 0, 3],
                      [-10, 0, 10],
                      [-3, 0, 3]])
        kernel_y = np.array([
                      [-3, -10, -3],
                      [0, 0, 0],
            [3, 10, 3],
                      ])
    else:
        raise ValueError("Unsupported filter method. Choose among 'sobel', 'prewitt', or 'scharr'.")
    
    # Convolve the image with the kernels (using symmetric padding)
    #############
    # CODE HERE #
    #############
    grad_x = convolve2d(image, kernel_x, mode='full', boundary='symm')
    grad_y = convolve2d(image, kernel_y, mode='full', boundary='symm')

    return grad_x, grad_y

#### HOG Feature Extraction and Visualization

**Description:**  
Extracts HOG (Histogram of Oriented Gradients) features from a grayscale image by:
- Dividing the image into cells of size `cell_size`.
- Computing an orientation histogram with `bin_size` bins (covering 0–180 degrees) for each cell.
- Normalizing groups of cells (blocks) using L2 norm over a block size of `block_size`.  
- **Parameters:**
  - `image`: 2D numpy array (grayscale).
  - `cell_size`: Size of each cell in pixels.
  - `bin_size`: Number of bins for the orientation histogram.
  - `block_size`: Number of cells per block for normalization.
  - `filter_method`: Differential filter method to compute gradients.
- **Returns:**
  - `feature_vector`: A flattened, normalized HOG feature vector for use in matching.
  - `cell_hist`: The orientation histogram for each cell, used for HOG visualization.
- **Usage:**  
  The HOG feature vector is used for sliding window matching (template matching) and the cell histograms are used for visualizing the HOG representation.


In [14]:
def hog_feature(image, cell_size, bin_size, block_size, filter_method):
    """
    Extract HOG features from an image.
    
    Parameters:
    - image: 2D numpy array (grayscale)
    - cell_size: size of each cell in pixels
    - bin_size: number of bins in the orientation histogram (covering 0-180 degrees)
    - block_size: number of cells per block for normalization
    - filter_method: differential filter method ('sobel', 'prewitt', or 'scharr')
    
    Returns:
    - feature_vector: normalized HOG feature vector (flattened for sliding window matching)
    - cell_hist: histogram for each cell (for HOG visualization)
    """
    # Compute gradients
    gx, gy = compute_gradients(image, method=filter_method)

    #############
    # CODE HERE #
    #############
    magnitude = np.sqrt(gx**2 + gy**2)
    # Compute angle in degrees and adjust to [0, 180)
    angle = np.arctan2(gy, gx) * 180 / np.pi

    h, w = image.shape
    n_cells_y = h // cell_size
    n_cells_x = w // cell_size
    cell_hist = np.zeros((n_cells_y, n_cells_x, bin_size))

    bin_width = 180 / bin_size

    # Calculate the histogram for each cell
    #############
    # CODE HERE #
    #############
    for i in range(n_cells_y):
        for j in range(n_cells_x):
            cell_magnitude = magnitude[i*cell_size:(i+1)*cell_size, j*cell_size:(j+1)*cell_size]
            cell_angle = angle[i*cell_size:(i+1)*cell_size, j*cell_size:(j+1)*cell_size]
            hist, _ = np.histogram(cell_angle, bins=bin_size, range=(0, 180), weights=cell_magnitude)
            cell_hist[i, j] = hist

    # Block normalization using L2 norm
    #############
    # CODE HERE #
    #############
    n_blocks_y = n_cells_y - block_size + 1
    n_blocks_x = n_cells_x - block_size + 1
    feature_vector = np.zeros((n_blocks_y, n_blocks_x, block_size * block_size * bin_size))
    for i in range(n_blocks_y):
        for j in range(n_blocks_x):
            block_hist = cell_hist[i:i+block_size, j:j+block_size].flatten() + 1e-6
            block_hist /= np.linalg.norm(block_hist, ord=2)
            feature_vector[i, j] = block_hist
    return feature_vector, cell_hist

**Description:**  
Visualizes the HOG features by overlaying red lines representing gradient orientations and magnitudes on the original image.  
- **Parameters:**
  - `original_image`: The original image (can be grayscale or RGB) used as the background.
  - `cell_hist`: The cell histograms obtained from the `hog_feature` function.
  - `cell_size`: Size of each cell in pixels.
  - `bin_size`: Number of histogram bins.
  - `output_file`: File name where the visualization will be saved.
- **Usage:**  
  This function helps to visually verify the HOG feature extraction by showing how the gradients are distributed across the image.


In [15]:
def visualize_hog(original_image, cell_hist, cell_size=8, bin_size=9, output_file='hog_visualization.png'):
    """
    Visualize HOG features overlaid on the original image.
    
    Parameters:
    - original_image: the original image (2D or 3D numpy array)
    - cell_hist: cell histograms from hog_feature function
    - cell_size: size of each cell in pixels
    - bin_size: number of histogram bins
    - output_file: file name to save the visualization
    
    The function overlays red lines representing HOG features on top of the original image.
    """
    #############
    # CODE HERE #
    #############
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(original_image, cmap='gray')

    # Define angles for bins
    angles = np.linspace(0, 180, bin_size, endpoint=False)

    # Iterate through each cell
    for i in range(cell_hist.shape[0]):
        for j in range(cell_hist.shape[1]):
            cell_center_y = i * cell_size + cell_size // 2
            cell_center_x = j * cell_size + cell_size // 2

            # Normalize the histogram for visualization
            hist = cell_hist[i, j]
            hist = hist / (np.max(hist) + 1e-6)

            # Draw lines for each bin
            for k, magnitude in enumerate(hist):
                angle = angles[k]
                rad = np.deg2rad(angle)
                dx = magnitude * cell_size * 0.5 * np.cos(rad)
                dy = magnitude * cell_size * 0.5 * np.sin(rad)

                ax.plot(
                    [cell_center_x - dx, cell_center_x + dx],
                    [cell_center_y - dy, cell_center_y + dy],
                    color='red', linewidth=1
                )

    plt.axis('off')
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
    plt.close()
    print(f"HOG visualization saved to {output_file}")

#### Sliding Window NCC Matching
**Description:**  
Converts an input image to grayscale.  
- **Parameters:**
  - `image`: A numpy array that can be either a 3D (RGB) or 2D (grayscale) image.
- **Returns:**
  - A 2D grayscale numpy array.
- **Usage:**  
  Ensures that images are in the correct grayscale format before processing with HOG feature extraction functions.

In [16]:
def convert_to_gray(image):
    """
    Convert an image to grayscale.
    
    Parameters:
    - image: numpy array; can be 2D (grayscale) or 3D (RGB)
    
    Returns:
    - 2D grayscale numpy array
    """
    if image.ndim == 3:
        return np.dot(image[...,:3], [0.299, 0.587, 0.114])
    else:
        return image

**Description:**  
Performs sliding window matching by computing the normalized cross-correlation (NCC) between the HOG features of a template image and each window in the target image.  
- **Parameters:**
  - `target_image`: Grayscale target image where face detection is performed.
  - `template_image`: Grayscale template image (face region) used for matching.
  - `cell_size`, `bin_size`, `block_size`, `filter_method`: Parameters for the HOG feature extraction.
  - `ncc_threshold`: Threshold for NCC score; windows with scores above this threshold are considered valid detections.
  - `step_size`: Pixel step size for moving the sliding window.
- **Returns:**
  - A list of candidate bounding boxes, each represented as `[x1, y1, x2, y2, ncc_score]`.
- **Usage:**  
  This function is used to find regions in the target image that match the template face based on HOG feature similarity.


In [17]:
def sliding_window_ncc(target_image, template_image, cell_size=8, bin_size=9, block_size=2, 
                       filter_method='sobel', ncc_threshold=0.5, step_size=4):
    """
    Perform sliding window matching using NCC on HOG features.
    
    Parameters:
    - target_image: target image (grayscale numpy array)
    - template_image: template image for face detection (grayscale numpy array)
    - cell_size, bin_size, block_size, filter_method: parameters for hog_feature
    - ncc_threshold: threshold for NCC score to consider a detection valid
    - step_size: pixel step for sliding window movement
    
    Returns:
    - List of candidate bounding boxes [x1, y1, x2, y2, ncc_score]
    """
    template_gray = convert_to_gray(template_image)
    template_feature, _ = hog_feature(template_gray, cell_size, bin_size, block_size, filter_method)
    template_feature = template_feature / (np.linalg.norm(template_feature) + 1e-6)
    
    t_h, t_w = template_gray.shape
    H, W = target_image.shape
    detections = []
    
    # Slide the window over the target image
    #############
    # CODE HERE #
    #############
    for y in range(0, H - t_h + 1, step_size):
        for x in range(0, W - t_w + 1, step_size):
            window = target_image[y:y+t_h, x:x+t_w]
            window_feature, _ = hog_feature(window, cell_size, bin_size, block_size, filter_method)
            window_feature = window_feature / (np.linalg.norm(window_feature) + 1e-6)
            ncc_score = np.sum(template_feature * window_feature)
            if ncc_score > ncc_threshold:
                detections.append([x, y, x + t_w, y + t_h, ncc_score])
    return detections

#### Non-Maximum Suppression
**Description:**  
Applies Non-Maximum Suppression (NMS) to remove overlapping detections. For overlapping boxes with an Intersection over Union (IoU) above the threshold, only the one with the highest NCC score is retained.  
- **Parameters:**
  - `detections`: List of candidate bounding boxes `[x1, y1, x2, y2, score]`.
  - `iou_threshold`: IoU threshold for suppression (default is 0.5).
- **Returns:**
  - A filtered list of bounding boxes after removing duplicates.
- **Usage:**  
  Helps in reducing duplicate detections, ensuring only the most confident detection for each face remains.


In [18]:
def non_max_suppression(detections, iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression to remove overlapping bounding boxes.
    
    Parameters:
    - detections: list of [x1, y1, x2, y2, score]
    - iou_threshold: IoU threshold for suppression (default 0.5)
    
    Returns:
    - List of filtered bounding boxes
    """
    #############
    # CODE HERE #
    #############
    if len(detections) == 0:
        return []
    boxes = np.array(detections)
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    scores = boxes[:, 4]
    areas = (x2 - x1) * (y2 - y1)
    order = scores.argsort()[::-1]
    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])
        w = np.maximum(0, xx2 - xx1)
        h = np.maximum(0, yy2 - yy1)
        inter = w * h
        iou = inter / (areas[i] + areas[order[1:]] - inter)
        order = order[np.where(iou <= iou_threshold)[0] + 1]
    filtered_bounding_boxes = boxes[keep]
    filtered_bounding_boxes = filtered_bounding_boxes[filtered_bounding_boxes[:, 4].argsort()[::-1]]
    filtered_bounding_boxes = filtered_bounding_boxes[:10]  # Keep top 10 detections
    # Filter out boxes with low scores
    filtered_bounding_boxes = filtered_bounding_boxes[filtered_bounding_boxes[:, 4] > 0.5]
    return filtered_bounding_boxes

#### Visualization of Detections on Original Image
**Description:**  
Visualizes detected faces by overlaying red bounding boxes and NCC scores on the original image. The NCC score is formatted to two decimal places and displayed on the image.  
- **Parameters:**
  - `original_image`: The original image (grayscale or RGB) where detections are visualized.
  - `detections`: List of bounding boxes `[x1, y1, x2, y2, score]`.
  - `output_file`: File name where the final detection visualization is saved.
- **Usage:**  
  Used to present the final detection results for evaluation or reporting.


In [19]:
def visualize_detections(original_image, detections, output_file='detections.png'):
    """
    Visualize detections by overlaying bounding boxes and NCC scores on the original image.
    
    Parameters:
    - original_image: the original image (numpy array; grayscale or RGB)
    - detections: list of bounding boxes [x1, y1, x2, y2, score]
    - output_file: file name to save the visualization
    
    The function draws red bounding boxes and overlays the NCC score (formatted to two decimals).
    """
    fig, ax = plt.subplots(1)
    if original_image.ndim == 2:
        ax.imshow(original_image, cmap='gray')
    else:
        ax.imshow(original_image)
    for det in detections:
        x1, y1, x2, y2, score = det
        width = x2 - x1
        height = y2 - y1
        # Draw bounding box
        rect = plt.Rectangle((x1, y1), width, height, edgecolor='red', facecolor='none', linewidth=2)
        ax.add_patch(rect)
        # Overlay NCC score text (formatted to two decimal places)
        ax.text(x1, y1, f"{score:.2f}", color='yellow', fontsize=12, backgroundcolor='black')
    plt.axis('off')
    plt.savefig(output_file, bbox_inches='tight', pad_inches=0)
    plt.close()
    print(f"Detection result saved to {output_file}")

#### Main Function

**Description:**  
The main function ties together the entire process:
1. Loads the template and target images and converts them to grayscale.
2. Extracts and visualizes HOG features on the target image.
3. Performs sliding window matching using NCC on HOG features.
4. Applies Non-Maximum Suppression to filter duplicate detections.
5. Visualizes the final detected faces on the original target image with bounding boxes and NCC scores.

**Notes for Students:**  
- **STUDENT:** Modify the file paths for `template.png` and `target.png` as needed for your environment.
- Adjust HOG parameters (e.g., `cell_size`, `bin_size`, `block_size`, `filter_method`) if required.


In [20]:
def main(template_path, 
         target_path,
         cell_size,
         bin_size,
         block_size,
         filter_method,
         ncc_threshold,
         step_size,
         iou_threshold,
         prefix=""):
    
    # Load images using PIL and convert them to grayscale
    template_img = np.array(Image.open(template_path).convert('L'))
    target_img = np.array(Image.open(target_path).convert('L'))
    
    # Compute HOG features for the target image and get cell histograms
    _, cell_hist = hog_feature(target_img, cell_size, bin_size, block_size, filter_method)
    
    # Visualize HOG features overlaid on the original target image
    visualize_hog(target_img, cell_hist, cell_size, bin_size, output_file=f'{prefix}hog_visualization_{filter_method}.png')
    
    # Extract HOG features for the template and perform sliding window NCC matching
    detections = sliding_window_ncc(target_img, 
                                    template_img, 
                                    cell_size, 
                                    bin_size, 
                                    block_size, 
                                    filter_method, 
                                    ncc_threshold, 
                                    step_size)
    print("Detections before NMS:", detections)
    
    # Apply Non-Maximum Suppression with an IoU threshold
    detections_nms = non_max_suppression(detections, iou_threshold)
    print("Detections after NMS:", detections_nms)
    
    # Visualize detections on the original target image
    visualize_detections(target_img, detections_nms, output_file=f'{prefix}detections_{filter_method}.png')
    

#### Run

In [21]:
img_template_path = 'template.png'
img_target_path = 'target.png'

main(template_path=img_template_path, 
     target_path=img_target_path,
     cell_size=8,
     bin_size=9,
     block_size=2,
     filter_method='sobel',
     ncc_threshold=0.5,
     step_size=4,
     iou_threshold=0.2)

HOG visualization saved to hog_visualization_sobel.png
Detections before NMS: [[52, 12, 102, 62, 0.5015010306966095], [56, 12, 106, 62, 0.5608640310088903], [52, 16, 102, 66, 0.5770243096829393], [56, 16, 106, 66, 0.5758718383282901], [92, 28, 142, 78, 0.5100211681400305]]
Detections after NMS: [[ 52.          16.         102.          66.           0.57702431]
 [ 92.          28.         142.          78.           0.51002117]]
Detection result saved to detections_sobel.png


In [22]:
main(template_path="my_template.jpg", 
     target_path="my_target.png",
     cell_size=8,
     bin_size=9,
     block_size=2,
     filter_method='sobel',
     ncc_threshold=0.5,
     step_size=4,
     iou_threshold=0.2,
     prefix="my_")

HOG visualization saved to my_hog_visualization_sobel.png
Detections before NMS: [[52, 8, 94, 73, 0.5098838360866832], [92, 8, 134, 73, 0.5736235926294332], [96, 8, 138, 73, 0.6102696424862118], [52, 12, 94, 77, 0.5086694955464912], [92, 12, 134, 77, 0.5526298507745347], [96, 12, 138, 77, 0.6296005880805722], [4, 24, 46, 89, 0.5918776195802739], [8, 24, 50, 89, 0.5461517088098178], [4, 28, 46, 93, 0.6419510345869286], [8, 28, 50, 93, 0.6637648873926292], [8, 32, 50, 97, 0.5196662293651202], [80, 68, 122, 133, 0.5219416201124566]]
Detections after NMS: [[  8.          28.          50.          93.           0.66376489]
 [ 96.          12.         138.          77.           0.62960059]
 [ 80.          68.         122.         133.           0.52194162]
 [ 52.           8.          94.          73.           0.50988384]]
Detection result saved to my_detections_sobel.png


In [23]:
main(template_path=img_template_path, 
     target_path=img_target_path,
     cell_size=8,
     bin_size=9,
     block_size=2,
     filter_method='prewitt',
     ncc_threshold=0.5,
     step_size=4,
     iou_threshold=0.2)
main(template_path=img_template_path, 
     target_path=img_target_path,
     cell_size=8,
     bin_size=9,
     block_size=2,
     filter_method='scharr',
     ncc_threshold=0.5,
     step_size=4,
     iou_threshold=0.2)

HOG visualization saved to hog_visualization_prewitt.png
Detections before NMS: [[52, 12, 102, 62, 0.5087380839094224], [56, 12, 106, 62, 0.5792685000371958], [52, 16, 102, 66, 0.5469475560766677], [56, 16, 106, 66, 0.5868960842017942], [92, 28, 142, 78, 0.5218105769808008], [96, 28, 146, 78, 0.5084696710398245], [92, 32, 142, 82, 0.5148152120050861]]
Detections after NMS: [[ 56.          16.         106.          66.           0.58689608]
 [ 92.          28.         142.          78.           0.52181058]]
Detection result saved to detections_prewitt.png
HOG visualization saved to hog_visualization_scharr.png
Detections before NMS: [[52, 12, 102, 62, 0.5033803333003901], [56, 12, 106, 62, 0.5593221449582887], [52, 16, 102, 66, 0.5728520271581732], [56, 16, 106, 66, 0.5834477601352548], [92, 28, 142, 78, 0.5096313488036295]]
Detections after NMS: [[ 56.          16.         106.          66.           0.58344776]
 [ 92.          28.         142.          78.           0.50963135]]
Dete